In [1]:
import pandas as pd
from modeling import feature_engineering, load_object, model_predict, save_csv
import numpy as np
import reverse_geocoder as revgc
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold
pd.options.display.float_format = '{:.4f}'.format

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
training='training'
label='interest_integer'
clf=xgb.XGBClassifier(n_jobs=-1,random_state=0,tree_method='gpu_hist')
df,columns,good_encoders,dp_table,df_tokenizer=load_object(clf.__class__.__name__+'_thesis')
dp_table

,columns,best value,encoders,time
after numeric feature selection,"[bathrooms, price, len_features, len_photos, d...",0.6321,{},00:05:36.217131
after TOP N features,"[bathrooms, price, len_features, len_photos, d...",0.6138,{},00:12:13.970803
after numeric features aggregated by categorical features,"[bathrooms, price, len_features, len_photos, d...",0.5533,{},00:39:36.258433
after numeric features interactions,"[bathrooms, price, len_features, len_photos, d...",0.5533,{},00:43:34.768650
after categorical feature encoding,"[bathrooms, price, len_features, len_photos, d...",0.5500,{'manager_id': <function CB_target_encoding at...,00:51:06.367922
after text feature tokenization plus PCA,"[bathrooms, price, len_features, len_photos, d...",0.5487,{'manager_id': <function CB_target_encoding at...,01:19:22.178376


In [3]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler


def objective(df,columns,training,label,encoder_dict,df_test_listingID,df_train_listingID,trial):
    params = {
        'n_estimators':trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        "min_child_weight": trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3, 1),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 10.0)
    }
    
    print(params)
    clf=xgb.XGBClassifier(n_jobs=-1,random_state=0,**params)
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf ,encoder_dict)
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))
    trial.set_user_attr('model_id',model_id)
    save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID)
    return lloss
n=100

sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,good_encoders,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)
x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_results.csv')

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



{'n_estimators': 694, 'learning_rate': 0.10167758525611789, 'max_depth': 2, 'min_child_weight': 4, 'subsample': 0.9289728088113784, 'gamma': 0.0599810287623786, 'colsample_bytree': 0.7364945877501806, 'tree_method': 'gpu_hist', 'lambda': 2.8802843563101424e-05, 'alpha': 4.7622419779375384e-06}
[I 2020-05-31 07:14:08,519] Finished trial#0 with value: 0.5466291014356818 with parameters: {'n_estimators': 694, 'learning_rate': 0.10167758525611789, 'max_depth': 2, 'min_child_weight': 4, 'subsample': 0.9289728088113784, 'gamma': 0.0599810287623786, 'colsample_bytree': 0.7364945877501806, 'tree_method': 'gpu_hist', 'lambda': 2.8802843563101424e-05, 'alpha': 4.7622419779375384e-06}. Best is trial#0 with value: 0.5466291014356818.
{'n_estimators': 482, 'learning_rate': 0.4337446433452351, 'max_depth': 12, 'min_child_weight': 2, 'subsample': 0.8958625190413323, 'gamma': 0.0001702783205227671, 'colsample_bytree': 0.6976311927657526, 'tree_method': 'gpu_hist', 'lambda': 2.13978305808933, 'alpha': 

In [3]:
df= pd.read_csv('ensemble_predictions_v1.csv')
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
columns=df.drop(columns=['target','training','listing_id']).columns
training='training'
label='target'

In [6]:
df

,listing_id,high_RandomForestClassifier_b4dd0f_0.579,medium_RandomForestClassifier_b4dd0f_0.579,low_RandomForestClassifier_b4dd0f_0.579,high_knn2Pipeline_3309d9_0.61,medium_knn2Pipeline_3309d9_0.61,low_knn2Pipeline_3309d9_0.61,high_LGBMClassifier_bb4c47_0.729,medium_LGBMClassifier_bb4c47_0.729,low_LGBMClassifier_bb4c47_0.729,...,medium_RandomForestClassifier_e597d7_0.569,low_RandomForestClassifier_e597d7_0.569,high_knn2Pipeline_5b4266_0.612,medium_knn2Pipeline_5b4266_0.612,low_knn2Pipeline_5b4266_0.612,high_knn2Pipeline_3a4dd0_0.609,medium_knn2Pipeline_3a4dd0_0.609,low_knn2Pipeline_3a4dd0_0.609,training,target
0,7170325,0.0980,0.4196,0.4825,0.0742,0.4758,0.4499,0.0007,0.4333,0.5660,...,0.4513,0.4592,0.0717,0.4792,0.4491,0.0707,0.4789,0.4505,1,2.0000
1,7092344,0.0879,0.3979,0.5143,0.1258,0.3480,0.5262,0.0018,0.8619,0.1363,...,0.4209,0.4985,0.1283,0.3396,0.5321,0.1245,0.3551,0.5204,1,1.0000
2,7158677,0.0222,0.2398,0.7380,0.0466,0.2200,0.7334,0.0003,0.1411,0.8586,...,0.2445,0.7400,0.0491,0.2302,0.7208,0.0430,0.2308,0.7263,1,2.0000
3,7211212,0.0496,0.2557,0.6947,0.0695,0.3450,0.5856,0.0003,0.0462,0.9534,...,0.2482,0.7033,0.0679,0.3283,0.6038,0.0679,0.3308,0.6013,1,2.0000
4,7225292,0.0050,0.0745,0.9205,0.0027,0.0836,0.9136,0.0000,0.0086,0.9913,...,0.0495,0.9486,0.0038,0.0792,0.9170,0.0030,0.0752,0.9218,1,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124006,6928108,0.3900,0.4077,0.2023,0.2042,0.3356,0.4602,0.6526,0.1854,0.1619,...,0.4395,0.1854,0.2113,0.3321,0.4566,0.2163,0.3274,0.4563,0,nan
124007,6906674,0.0142,0.1728,0.8131,0.0086,0.1994,0.7920,0.0001,0.1223,0.8776,...,0.1649,0.8258,0.0113,0.1887,0.8000,0.0095,0.1941,0.7963,0,nan
124008,6897967,0.0130,0.0788,0.9082,0.0025,0.0221,0.9754,0.0000,0.0021,0.9979,...,0.0482,0.9445,0.0000,0.0189,0.9811,0.0027,0.0216,0.9757,0,nan
124009,6842183,0.0016,0.0316,0.9668,0.0000,0.0508,0.9492,0.0000,0.0023,0.9977,...,0.0233,0.9764,0.0000,0.0528,0.9472,0.0000,0.0531,0.9469,0,nan


In [2]:
import pandas as pd
df= pd.read_csv('ensemble_predictions_v1.csv')
df

,listing_id,high_XGBClassifier_b7bffc_0.537,medium_XGBClassifier_b7bffc_0.537,low_XGBClassifier_b7bffc_0.537,high_LGBMClassifier_a2f6d9_0.538,medium_LGBMClassifier_a2f6d9_0.538,low_LGBMClassifier_a2f6d9_0.538,high_LGBMClassifier_4265b5_0.537,medium_LGBMClassifier_4265b5_0.537,low_LGBMClassifier_4265b5_0.537,...,medium_LGBMClassifier_4dfd2f_0.543,low_LGBMClassifier_4dfd2f_0.543,high_XGBClassifier_f5884f_0.536,medium_XGBClassifier_f5884f_0.536,low_XGBClassifier_f5884f_0.536,high_XGBClassifier_570512_0.547,medium_XGBClassifier_570512_0.547,low_XGBClassifier_570512_0.547,training,target
0,7170325,0.015991,0.383648,0.600361,0.014799,0.288568,0.696633,0.024960,0.374595,0.600445,...,0.342669,0.646591,0.025680,0.419893,0.554427,0.032783,0.403438,0.563779,1,2.0
1,7092344,0.076603,0.442511,0.480887,0.082838,0.589275,0.327887,0.062741,0.622731,0.314528,...,0.662088,0.295039,0.069985,0.462028,0.467987,0.077435,0.355732,0.566833,1,1.0
2,7158677,0.012489,0.201786,0.785725,0.009752,0.300312,0.689937,0.012250,0.250385,0.737365,...,0.274707,0.718715,0.014403,0.235764,0.749833,0.023209,0.226445,0.750346,1,2.0
3,7211212,0.031132,0.229040,0.739828,0.020930,0.199562,0.779508,0.036899,0.255091,0.708011,...,0.244794,0.736652,0.040647,0.292849,0.666504,0.041665,0.366000,0.592334,1,2.0
4,7225292,0.001184,0.055771,0.943044,0.002251,0.066077,0.931672,0.003725,0.064699,0.931577,...,0.052655,0.945838,0.001744,0.046608,0.951648,0.001680,0.069370,0.928950,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124006,6928108,0.660341,0.223336,0.116323,0.625436,0.248864,0.125700,0.466859,0.379503,0.153639,...,0.294405,0.167000,0.619873,0.256706,0.123421,0.652127,0.261873,0.086000,0,NaN
124007,6906674,0.009475,0.257527,0.732999,0.007384,0.218222,0.774394,0.005954,0.214454,0.779591,...,0.245726,0.749300,0.007982,0.233083,0.758935,0.011999,0.247545,0.740456,0,NaN
124008,6897967,0.000316,0.014360,0.985324,0.000709,0.024006,0.975284,0.001355,0.026357,0.972288,...,0.016511,0.982933,0.000590,0.015106,0.984304,0.000654,0.022459,0.976887,0,NaN
124009,6842183,0.000625,0.029586,0.969789,0.000619,0.036247,0.963134,0.001397,0.034048,0.964555,...,0.031625,0.967827,0.000772,0.035862,0.963366,0.000529,0.037800,0.961671,0,NaN


In [ ]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler
import pandas as pd
from modeling import feature_engineering, load_object, model_predict, save_csv
import numpy as np
import reverse_geocoder as revgc
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold
pd.options.display.float_format = '{:.4f}'.format
df= pd.read_csv('ensemble_predictions_v1.csv')
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
columns=df.drop(columns=['target','training','listing_id']).columns
training='training'
label='target'

def objective(df,columns,training,label,df_test_listingID,df_train_listingID,trial):
    params = {
        'n_estimators':trial.suggest_int('n_estimators',300, 351),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.04),
        'max_depth': trial.suggest_int('max_depth', 4, 5), #YMEN NA 8
        "min_child_weight": trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 10.0)
    }
    
    print(params)
    clf=xgb.XGBClassifier(n_jobs=-1,random_state=0,**params)
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf)
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))
    trial.set_user_attr('model_id',model_id)
    save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID,dir='Thesis_predictions_ensemble/')
    return lloss
n=25

sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)
x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_results_XGB_POKUS_VYMAZ.csv')

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



{'n_estimators': 344, 'learning_rate': 0.022747204170935315, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.9431782470491027, 'gamma': 0.0599810287623786, 'colsample_bytree': 0.8870691090357917, 'tree_method': 'gpu_hist', 'lambda': 2.8802843563101424e-05, 'alpha': 4.7622419779375384e-06}
[I 2020-06-05 18:03:50,638] Finished trial#0 with value: 0.5267994245895319 with parameters: {'n_estimators': 344, 'learning_rate': 0.022747204170935315, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.9431782470491027, 'gamma': 0.0599810287623786, 'colsample_bytree': 0.8870691090357917, 'tree_method': 'gpu_hist', 'lambda': 2.8802843563101424e-05, 'alpha': 4.7622419779375384e-06}. Best is trial#0 with value: 0.5267994245895319.
{'n_estimators': 324, 'learning_rate': 0.038034945056526645, 'max_depth': 4, 'min_child_weight': 2, 'subsample': 0.9166900152330658, 'gamma': 0.0001702783205227671, 'colsample_bytree': 0.8704133683281796, 'tree_method': 'gpu_hist', 'lambda': 2.13978305808933, 'alpha